# Simulus Tutorial

This tutorial describes the basic functions of simulus. All examples shown in this tutorial can be found under the `examples/basics` directory in the simulus source-code distribution.

Simulus is a process-oriented discrete-event simulator written in Python. That's quite a mouthful. Let's first break it down a bit what it really means. 

## 1.  Discrete-Event Simulation and Process-Oriented World View

A discrete-event simulator models the world as a sequence of events that happen in discrete time. The simulator basically processes these events in order. Such a simulator would keep a simulation clock, which advances its time by leaps and bounds in accordance with the time of the events being processed along the way.

For example, a professor starts the day with a cup of coffee at 4 AM, reads for 2 hours, has breakfast, takes shower, and then drives to school at 7:30 AM. The discrete-event simulator for the professor's day will go through a sequence of events: "wake up" event at 4:00, "start coffee" event at 4:05, "start read" event at 4:10, "finish coffee" event at 4:20, "finish read" event at 6:10, breakfast event at 6:30, shower event at 6:50, "start driving" event at 7:30, etc. The clock of the simulator in this case will go through a sequence of distinct values: 4:00, 4:05, 4:10, ... You got the gist. 

Discrete-event simulation is a very powerful modeling method. You can think of the entire world, including the activities of all professors, students, and all people, as a sequence of events that happen over time. The simulator's job is to play out the events in order. By doing so, we get better understanding of the world.

Simulus is a "process-oriented" discrete-event simulator. What it means is that in addition to describing the world as a bunch of events, one can also model it using processes. As an example, suppose the world consists of many professors, students, and people. Previously we described the schedule of one professor. A different person would have a different schedule, i.e., following a different sequence of events. Using process-oriented simulation, every person in this system can be modeled as a separate process. 

Note that people interact in this world. For example, if everyone starts driving to school or work in the morning at the same hour, most people would get delayed since congestion would happen on the road. Also, if a student needs to meet with another student at a coffee shop. They would not be able to carry on with their next task (say, to work on their joint project) until both of them can get to the coffee shop. Simulus provides the needed support for creating and managing the processes, making it easier for one to model the world.

## 2.  How Simulus Works

Simulus works in two ways. One way is through events. You can schedule events and process them. We call it *direct event scheduling*. The other way is through processes. You can create processes and have them run and interact. We call it *process scheduling*. Of course, there's a third way, by combining them and having them both.

### 2.1  Direct Event Scheduling

In simulus, an event is simply a function to be invoked at a designated time. This can be illustrated using the following hello-world example.

#### 2.1.1  The Hello-World Example

In [1]:
# %load "../examples/basics/helloworld.py"
import simulus

def print_message():
    print("Hello world at time", sim.now)
    
sim = simulus.simulator()
sim.sched(print_message, until=10)
sim.run()


Hello world at time 10


In order to use the simulator, we need to first import the `simulus` module and then create a simulator using the `simulator()` method. A simulator maintains an event list where all events will be stored and sorted in their timestamp order. A simulator also keeps the time, which gets advanced while executing the events. One can find out the current time of the simulator by inspecting the `now` variable of the simulator.

In the example, the function `print_message()` is scheduled to run at time 10. We schedule the function using the `sched()` method of the simulator, and passing the name of the function and using the `until` argument to specify the time at which the function should be invoked. In discrete-event simulation terminology, the function is also called an *event handler*. An event handler in simulus can be any function.

To run the simulation, we simply use the `run()` method of the simulator. Without an argument, the `run()` method will process *all* events on the event list and returns only when there are no more events left.

#### 2.1.2  Passing Arguments to Event Handlers

The next example shows how to pass arguments between `sched()` and the event handler. 

In [2]:
# %load "../examples/basics/passargs.py"
import simulus

def print_params_1(a, b, c="named"):
    print("print_params_1(a=%r, b=%r, c=%r) invoked at time %g" %
          (a, b, c, sim.now))

def print_params_2(mysim, a, b, *args, c="named", **kwargs):
    print("print_params_2(a=%r, b=%r, args=%r, c=%r, kwargs=%r) invoked at time %g" %
          (a, b, args, c, kwargs, mysim.now))

class myclass:
    def __init__(self, a):
        self.a = a

    def print_params_3(self, b, c):
        print("print_params_3(a=%r, b=%r, c=%r) invoked at %g" %
              (self.a, b, c, sim.now))
    
sim = simulus.simulator()
sim.sched(print_params_1, "hello", 100, until=10)
sim.sched(print_params_2, sim, "hello", 100, "yes", "no", arg1=True, arg2=2, c=1, until=20)
cls = myclass(10)
sim.sched(myclass.print_params_3, cls, 11, 12, until=30)
sim.run()


print_params_1(a='hello', b=100, c='named') invoked at time 10
print_params_2(a='hello', b=100, args=('yes', 'no'), c=1, kwargs={'arg1': True, 'arg2': 2}) invoked at time 20
print_params_3(a=10, b=11, c=12) invoked at 30


In the hello-world example, we did not pass any argument to the `print_params()` function. The `sim` variable is module scoped. That is, as long as the function stays in the same module where `sim` is defined, it'll be fine. But this would not work in a more complicated scenario, when we develop a model that spreads into multiple python modules, or when the functions need to have paramesters.

In this example, we schedule three functions: `print_params_1()` at time 10, `print_params_1()` at time 20, and `print_params_3()` at time 30. We can pass aguments from `sched()` to the event handler by placing the arguments right after the function name (the function name has to be the first argument of `sched()`). We can take advantage of Python's ability to take both positional arguments and keyworded arguments. In Python, the single-asterisk form of `*args` can be used as a parameter to take a non-keyworded variable-length argument list to functions. The double asterisk form of `**kwargs` is used to take a keyworded, variable-length argument dictionary to a function. 

In `print_params_1()`, the variables `a` and `b` take the positional arguments, and `c` takes a keyworded argument. In `print_params_2()`, the variables `mysim`, `a`, and `b` take the first three positional arguments: sim, "hello", and 100. The additional positional arguments, "yes" and "no", are placed into the list `args`. The variable `c` is a keyworded argument. The rest of the keyworded arguments are placed into the dictionary `kwargs`. `print_params_3()` is a method inside `myclass`. As a method, the first argument must be an instance of the class. Consequently, when we schedule a method as the event handler we need to have the class instance as the first positional argument.

The `sched()` function takes its own arguments. They are all keyworded arguments, such as `until`. The `sched()` function will first filter out the keyworded arguments for itself, before scheduling the user intended function to be invoked at the designated time. All positional arguments are for the event handler.

In Python, the parameters of a function need to occur in a particular order: first, the formal positional arguments, then `*args`, followed by the keyworded arguments, and finally `**kwargs`. That's exactly what we did for the `print_params_2` function.

#### 2.1.3  The Life of a Professor

In simulus, one can of course schedule more functions to run within an event handler. The whole point of simulation is to help capture and examine the complicated logic or processes of the world (or system) as the events unfold. 

In the next example, we show the "complicated" life of a professor.

In [3]:
# %load "../examples/basics/professor.py"
import simulus

from time import gmtime, strftime
def strnow():
    return strftime("%H:%M:%S", gmtime(sim.now))

def wake_up():
    print("professor wakes up at "+strnow())
    sim.sched(start_coffee, offset=5*60) # 5 minutes from now
    sim.sched(breakfast, offset=2*3600+30*60) # 2 hours and 30 minutes from now
    sim.sched(shower, offset=2*3600+50*60) # 2 hours and 50 minutes from now
    sim.sched(leave, offset=3*3600+30*60) # 3 hours and 30 minutes from now
    
def start_coffee():
    print("professor starts drinking coffee at "+strnow())
    sim.sched(finish_coffee, offset=15*60) # 15 minutes from now
    sim.sched(start_read, offset=5*60) # 5 minutes from now

def finish_coffee():
    print("professor finishes drinking coffee at "+strnow())
    
def start_read():
    print("professor starts reading at "+strnow())
    sim.sched(finish_read, offset=2*3600) # 2 hours from now
    
def finish_read():
    print("professor finishes reading at "+strnow())

def breakfast():
    print("professor breakfasts at "+strnow())

def shower():
    print("professor shows at "+strnow())

def leave():
    print("professor leaves home and drives to school at "+strnow())
    sim.sched(arrive, offset=45*60) # 45 minutes from now

def arrive():
    print("professor arrives at school at "+strnow())

def meeting1():
    print("professor has first meeting at "+strnow())

def meeting2():
    print("professor has second meeting at "+strnow())

sim = simulus.simulator()
sim.sched(wake_up, until=4*3600) # 4:00
sim.sched(meeting1, until=9*3600) # 9:00
sim.sched(meeting2, until=10*3600) # 10:00
sim.run()


professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor shows at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 08:15:00
professor has first meeting at 09:00:00
professor has second meeting at 10:00:00


The time is represented as a floating point number in simulus. In this example, we use time in seconds starting from midnight. To make things a bit more comprehensible, we use the standard time module in Python to turn the seconds into hours and minutes in the printout.

#### 2.1.4  Canceling and Rescheduling Events

So far, the professor's schedule is rather boring. In real life, we make appointments and we cancel appointments. We also reschedule appointments. Let's make things a bit more interesting for the professor's life. Let's assume that today there's a big traffic jam. So instead of taking only 45 minutes, the professor takes 2 hours 45 minutes to get to school. Obviously the professor will miss the first meeting and get late for the second meeting. So she cancels the first meeting and reschedules the second meeting. 

To do that we need to use the return value from the `sched()` method. It is actually a reference to an event created by the sched() method. Event in simulus is an opaque object. That is, we can use the event reference to cancel or reschedule the event, but we should not directly access the variables and methods within the event class. In this example, we call the simulator's `cancel()` and `resched()` methods and pass the event as the argument. The professor cancels and reschedules the events when she is about to leave home (assuming she predicts that the traffic jam is about to happen).

Another difference of this example from the previous one is that we make professor's waking up a daily repeatable event (with a 24 hour interval). In this case, the professor always gets up at 4 AM every day and carries out the same set of activities. If we `run()` the simulation without an argument, it would never return as there will always be events on the event list. Because of that, in this example, we call `run()` with a simulation end time using the named argument `until`. We run simulation for a duration of 3 days.

In [4]:
# %load "../examples/basics/professor-flex.py"
import simulus

from time import gmtime, strftime
def strnow():
    return strftime("%H:%M:%S", gmtime(sim.now))

def wake_up():
    print("professor wakes up at "+strnow())
    sim.sched(start_coffee, offset=5*60) # 5 minutes from now
    sim.sched(breakfast, offset=2*3600+30*60) # 2 hours and 30 minutes from now
    sim.sched(shower, offset=2*3600+50*60) # 2 hours and 50 minutes from now
    sim.sched(leave, offset=3*3600+30*60) # 3 hours and 30 minutes from now
    
def start_coffee():
    print("professor starts drinking coffee at "+strnow())
    sim.sched(finish_coffee, offset=15*60) # 15 minutes from now
    sim.sched(start_read, offset=5*60) # 5 minutes from now

def finish_coffee():
    print("professor finishes drinking coffee at "+strnow())
    
def start_read():
    print("professor starts reading at "+strnow())
    sim.sched(finish_read, offset=2*3600) # 2 hours from now
    
def finish_read():
    print("professor finishes reading at "+strnow())

def breakfast():
    print("professor breakfasts at "+strnow())

def shower():
    print("professor showers at "+strnow())

def leave():
    print("professor leaves home and drives to school at "+strnow())
    if sim.now < 24*3600:
        # traffic jam at the first day
        sim.sched(arrive, offset=2*3600+45*60) # 2 hours and 45 minutes from now
        # the two meetings are only at the first day
        sim.cancel(e1)
        sim.resched(e2, until=11*3600) # 11:00
    else:
        # no traffic jam in the following days
        sim.sched(arrive, offset=45*60) # 45 minutes from now

def arrive():
    print("professor arrives at school at "+strnow())

def meeting1():
    print("professor has first meeting at "+strnow())

def meeting2():
    print("professor has second meeting at "+strnow())

sim = simulus.simulator()
sim.sched(wake_up, until=4*3600, repeat_intv=24*3600) # 4:00 
e1 = sim.sched(meeting1, until=9*3600) # 9:00
e2 = sim.sched(meeting2, until=10*3600) # 10:00
sim.run(until=72*3600)


professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor showers at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 10:15:00
professor has second meeting at 11:00:00
professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor showers at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 08:15:00
professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor sho

#### 2.1.5  More on Running Simulators

By default, a simulator starts at time zero. Instead, we can use the `init_time` argument to change the start simulation time when we create the simulator. 

A simulator can also have an optional name using the `name` argument. If specified, the simulator's name should be unique. In this case, one can use the `get_simulator()` function to retrieve the simulator using the name. If the name is not unique, a simulator created later with the same name may overwrite the earlier one, which will no longer be retrievable using the name. A simulator's name is optional. A simulator can be created without a name. In this case, one cannot use the `get_simulator()` function to retrieve it.

All the previous examples create only one simulator at a time. In fact, one can simultaneously run multiple simulators in simulus, each maintaining its own event list and its own simulation time. If multiple simulators are created, they will all run independently. The event handlers at different simulators are invoked in separate timelines. That is, the current time at one simulator has no relation to the current time of another simulator. This is the default behavior. Simulators can also be time synchronized (using the `sync()` function). We will return to this topic in the advanced tutorial when we discuss the parallel and distributed simulation support of simulus.

Each simulator processes its own events when we call the `run()` method. We can specify either `offset` or `until` (but not both) when we call `run()`, in which case the simulator will process all events with timestamps no larger than the designated time. The `offset` argument specifies a relative time from the current time. The `until` argument specifies the absolute time. When the method returns, the current time of the simulator will be advanced to the designated time. If both `offset` and `until` are ignored, the simulator will run as long as there are events on the event list. Be careful in this case, a simulator may run forever for some models as new events are generated repeatedly when the event handlers are invoked (like in the previous example).

We can also step through simulation processing one event at a time. This is achieved using the simulator's `step()` method, which processes the next event on the event list and advances the simulation clock to the time of the event. If there is no event available on the event list, the method has no effect. 

To determine the time of the next event, we can use the `peek()` method of the simulator. We can also list all future events of a simulator using the `show_calendar()` method. This method is supposedly used only for debugging purposes. Listing all events on an event list can be an expensive operation.

The following example illustrates some of the functions we just metioned in this section.

In [5]:
# %load "../examples/basics/twosims.py"
import simulus

def handle(sim):
    print("'%s' handles event at time %g" % (sim.name, sim.now))

sim1 = simulus.simulator(name="sim1", init_time=100)
sim2 = simulus.simulator(name="sim2", init_time=-100)

for i in range(5, 100, 20):
    sim1.sched(handle, sim1, offset=i)
for i in range(5, 200, 30):
    sim2.sched(handle, sim2, offset=i)
sim1.show_calendar()
sim2.show_calendar()

while True:
    t1, t2 = sim1.peek(), sim2.peek()
    if t1 < simulus.infinite_time:
        sim1.step()
    if t2 < simulus.infinite_time:
        sim2.step()
    if t1 == simulus.infinite_time and \
       t2 == simulus.infinite_time:
        break


list of all future events (num=5) at time 100 on simulator sim1:
  105: dir_evt=handle() 
  125: dir_evt=handle() 
  145: dir_evt=handle() 
  165: dir_evt=handle() 
  185: dir_evt=handle() 
list of all future events (num=7) at time -100 on simulator sim2:
  -95: dir_evt=handle() 
  -65: dir_evt=handle() 
  -35: dir_evt=handle() 
  -5: dir_evt=handle() 
  25: dir_evt=handle() 
  55: dir_evt=handle() 
  85: dir_evt=handle() 
'sim1' handles event at time 105
'sim2' handles event at time -95
'sim1' handles event at time 125
'sim2' handles event at time -65
'sim1' handles event at time 145
'sim2' handles event at time -35
'sim1' handles event at time 165
'sim2' handles event at time -5
'sim1' handles event at time 185
'sim2' handles event at time 25
'sim2' handles event at time 55
'sim2' handles event at time 85


In this example, we create two simulators, one starting from time 100 and the other from time -100. (Yes, we can use negative simulation time!) For one simulator, we schedule a function to be invoked 5 times at regular time intervals with a gap of 20 and starting at an offset of 5 from the simulator's start time. For the other simulator, we schedule the same function to be invoked 7 times at regular time intervals with a gap of 30 and starting at an offset of 5 from that simulator's start time. We then step through the scheduled events of the two simulators, processing one event at a time for each simulator in an alterative fashion as long as they have events.

### 2.2  Process Scheduling

Another way to model the world is to use simulation processes. Conceptually, a process is just a thread of control, which is similar to sequentially running through a prgram, that constitutes a bunch of statements, including if-branches, while-loops, and function calls. During its execution, a simulation process can be blocked, either sleeping for some time, or requesting for some resource that is currently unavailable. The process will resume execution when the specified time has passed or after the resource blocking condition has been removed. 

Simulus implements the simulation processes as "coroutines" (also known as "micro-threads" or "tasklets"), using Python's greenlet package.

#### 2.2.1  The Hello-World Example Using Process

The following is a modified hello-world example, which now uses a process.

In [6]:
# %load "../examples/basics/helloworld-proc.py"
import simulus

def print_message():
    for _ in range(10):
        print("Hello world at time", sim.now)
        sim.sleep(1)
    
sim = simulus.simulator()
sim.process(print_message, until=10)
sim.run()


Hello world at time 10
Hello world at time 11
Hello world at time 12
Hello world at time 13
Hello world at time 14
Hello world at time 15
Hello world at time 16
Hello world at time 17
Hello world at time 18
Hello world at time 19


In this example, the function print_message() is the starting function of a process. We create the process using the `process()` method of the simulator and schedule the process to run at time 10 using the `until` argument. 

As with the earlier example that uses the `sched()` method, we can pass aguments to the starting function when we create the process, either as positional arguments, or keyworded arguments, or both. They will be passed as expected to to the starting function of the process when it starts running at the designated time.

In this modified helloworld example, the process loops for 10 times; at each iteration, it prints out a hello message and sleeps for 1 second. The `sleep()` method takes one argument: either an `offset` argument, which gives the amount of time the process will be put to sleep, or an `until` argument, which indicates the time at which the process should resume execution. We don't need to explicitly name the argument as `offset` as in this example; it's the default positional argument.

#### 2.2.2  The Professor's Life as a Process

Now we return to the previous example and use a process to simulate the professor's complicated life.

In [7]:
# %load "../examples/basics/professor-proc.py"
import simulus

from time import gmtime, strftime
def strnow():
    return strftime("%H:%M:%S", gmtime(sim.now))

def prof_life():
    while True:
        start_of_the_day = sim.now
        
        sim.sleep(4*3600) # 4 hours from midnight
        print("professor wakes up at "+strnow())
        
        sim.sleep(offset=5*60) # 5 minutes from now
        print("professor starts drinking coffee at "+strnow())
    
        sim.sleep(offset=5*60) # 5 minutes from now
        print("professor starts reading at "+strnow())

        sim.sleep(offset=(15-5)*60) # 15 minus 5 minutes from now
        print("professor finishes drinking coffee at "+strnow())

        sim.sleep(offset=2*3600-10*60) # 2 hours minus 10 minutes from now
        print("professor finishes reading at "+strnow())

        sim.sleep(until=start_of_the_day+6*3600+30*60) # 6:30
        print("professor breakfasts at "+strnow())
        
        sim.sleep(until=start_of_the_day+6*3600+50*60) # 6:50
        print("professor showers at "+strnow())

        sim.sleep(until=start_of_the_day+7*3600+30*60) # 7:30
        print("professor leaves home and drives to school at "+strnow())

        if sim.now < 24*3600:
            # traffic jam at the first day
            sim.sleep(offset=2*3600+45*60) # 2 hours and 45 minutes from now
            print("professor arrives at school at "+strnow())

            if sim.now < 9*3600:
                # if arrives before the 9 o'clock, attend both meetings
                sim.sleep(until=9*3600)
                print("professor has first meeting at "+strnow())

                sim.sleep(until=10*3600)
                print("professor has second meeting at "+strnow())
            else:
                # if late, no the first meeting and resched the second
                sim.sleep(until=11*3600)
                print("professor has second meeting at "+strnow())
        else:
            # no traffic jam in the following days
            sim.sleep(offset=45*60) # 45 minutes from now
            print("professor arrives at school at "+strnow())

        # the rest of the day is a blur for the professor
        rest_of_the_day(start_of_the_day)

def rest_of_the_day(start):
    # sleep until the start of the next day
    sim.sleep(until=start+24*3600)
            
sim = simulus.simulator()
sim.process(prof_life) 
sim.run(until=72*3600)


professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor showers at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 10:15:00
professor has second meeting at 11:00:00
professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor showers at 06:50:00
professor leaves home and drives to school at 07:30:00
professor arrives at school at 08:15:00
professor wakes up at 04:00:00
professor starts drinking coffee at 04:05:00
professor starts reading at 04:10:00
professor finishes drinking coffee at 04:20:00
professor finishes reading at 06:10:00
professor breakfasts at 06:30:00
professor sho

Rather than having many event handlers as in the earlier example, the professor's life now becomes one process. It's one big loop and one iteration of the loop represents one day of the professor's life. At each iteration, the professor calls `sleep()` to advance the time. 

There is one caveat for using the `sleep()` method: it can only be called when a process is running, in the starting function of the process or in a function directly or indirectly invoked by the starting function. Otherwise, simulus will raise an exception.

The professor goes through all the chores of the day. At the end of those chores, she calls the function `rest_of_the_day()` to simulate all the activities she would take for the rest of the day. We are still in the same process. The function sleeps until the beginning of the next day and returns. The loop continues to the next iteration for the next day. 

#### 2.2.3  Processes Everywhere

Of course, the world does not have just the professor in it. There are other professors and students, and many other people as well. If we simulate a multi-agent system, for example, each agent can be treated an automic entity with its own agenda. So it's natural to simulate such a system using processes to represent the independent agents. Each agent may also use multiple processes. Recall that professor in the previous example actually has coffee and reads at the same time. Conceptually the two overlapping activities can be modeled as separate processes.

Our world is a complicated world. And processes interact. The agents in a multi-agent system may communicate and synchronize with one another and they compete for resources. Simulus provides the necessary facilities for processes to fulfill these functions. We will deal with the more complicated issues later.

For now, let's examine a simple example showing the creation and execution of multiple processes, and a simple way to synchronize them (by waiting for the completion of the processes).

In [8]:
# %load "../examples/basics/homework.py"
import simulus

from random import seed, expovariate, gauss
seed(12345)

def student(student_id):
    print("student %d starts to work at %g" % 
          (student_id, sim.now))
    sim.sleep(gauss(30, 5)) # work on part one
    sim.sleep(expovariate(1/10.)) # take some rest
    sim.sleep(gauss(60, 10)) # work on part two
    print("student %d finishes working at %g" % 
          (student_id, sim.now))

def homework():
    print("homework assigned at "+str(sim.now))
    # five students working on the homework each starts at a random 
    # time (exponentially distributed with mean of 10)
    students = []
    for i in range(5):
        s = sim.process(student, i, offset=expovariate(1/10.))
        students.append(s)
    # wait for all student processes to complete
    sim.wait(students)
    print("last student finishes homework at "+str(sim.now))

sim = simulus.simulator()
sim.process(homework, offset=10) 
sim.run()


homework assigned at 10
student 1 starts to work at 10.1022
student 3 starts to work at 13.5473
student 4 starts to work at 14.5952
student 0 starts to work at 15.3892
student 2 starts to work at 27.4415
student 4 finishes working at 108.348
student 1 finishes working at 109.514
student 2 finishes working at 111.156
student 0 finishes working at 122.177
student 3 finishes working at 141.183
last student finishes homework at 141.18344556219594


In this example, a `homework` process is created and scheduled to run at time 10, at which the homework is assigned. Five students, each represented as a separate process created by the `homework` process, will work on the assignment. Each `student` process starts at some random time, sleep for some random time to represent the student's working on the first part of the homework, sleep again for some random time representing the student's taking a break, and finally sleep for some random time representing the student's working on the second part of the homework. The `homework` process waits for all the student processes to finish using the magical `wait()` method. We will explain the magic behind this method later.

This example is the first time in this tutorial we deal with random numbers. It needs a bit explanation. We use Python's `random` package. In particular, we use the `seed()` method to fix a seed for the default random number generator, so that every time we run this example, we get the same results. In simulation, we call the numbers generated by the `random` package *pseudo-random numbers*. The numbers indeed look random, but they are not. They follow a fixed sequence seemingly unpredictably. As long as we start the random sequence with a fixed seed, the sequence is guaranteed be the same.

In this example, we use random numbers from two distributions. The `expovariate()` method generates an exponentially distributed random number. The argument to the method is one divided by the desired mean of the distribution. The `gauss()` method generates a Gaussian (normally) distributed random number. The arguments to the method is the desired mean and standard deviation of the normal distribution. 

## 3.  Inter-Process Communication

Processes often need to interact with one another in order to achieve tasks. In the previous homework example, we saw one simplest way to communicate among the processes: the `homework` process waits for the completion of all `student` processes. Simulus provides a rich set of mechanisms to support inter-process communication. We start by discussing the two basic primitive methods designed specifically for the simulation processes to synchronize and communicate with one another. One is called a trap and the other is called a semaphore.

### 3.1  Traps

A trap is a one-time signaling mechanism for inter-process communication. A trap has three states. It's "unset" when the trap is first created and and nothing has happended to it. It's "set" when one or more processes are waiting for the trap to be triggered. It turns to "sprung" when the trap has been triggered, after which there will be no more processes waiting for the trap.

The life of a trap is as follows. A trap starts with the "unset" state when it's created. When a process waits for the trap, the trap goes to "set", at which state more processes may wait on the trap and the trap remains in the same "set" state. When a process triggers the trap and if the trap is in the "set" state, *all* processes waiting on the trap will be unblocked to resume execution (it's guaranteed there is at least one waiting process when the trap is in the "set" state). The trap will then transition into the "sprung" state. When a process triggers the trap which is in the "unset" state, the trap will just transition to the "sprung" state (since there are no processes currently waiting on the trap). If the trap has "sprung", further waiting on trap will be considered as an no-op; that is, the processes trying to wait on a "sprung" trap will not be suspended. 

#### 3.1.1  One-Time Signaling

A trap is a simple signaling mechanism. One or more processes can wait on a trap. Another process can trigger the trap to release all the waiting processes at once.  A trap is also for one-time use only. Once sprung, a trap must not be triggered any more, or simulus will raise an exception. That is, if a process wants to send multiple signals to other processes, one would have to multiple traps. 

The following example shows a simple use case for traps.

In [9]:
# %load "../examples/basics/onetrap.py"
import simulus

def p(idx):
    if idx > 0:
        print("p%d starts at %g and waits on trap" % (idx, sim.now))
        t.wait()
        print("p%d resumes at %g" % (idx, sim.now))
    else:
        print("p%d starts at %g" % (idx, sim.now))
        sim.sleep(5)
        print("p%d triggers the trap at %g" % (idx, sim.now))
        t.trigger()

sim = simulus.simulator()
t = sim.trap()
for i in range(10):
    sim.process(p, i, offset=10+i)
sim.run()


p0 starts at 10
p1 starts at 11 and waits on trap
p2 starts at 12 and waits on trap
p3 starts at 13 and waits on trap
p4 starts at 14 and waits on trap
p5 starts at 15 and waits on trap
p0 triggers the trap at 15
p1 resumes at 15
p2 resumes at 15
p3 resumes at 15
p4 resumes at 15
p5 resumes at 15
p6 starts at 16 and waits on trap
p6 resumes at 16
p7 starts at 17 and waits on trap
p7 resumes at 17
p8 starts at 18 and waits on trap
p8 resumes at 18
p9 starts at 19 and waits on trap
p9 resumes at 19


In this example, we create a trap using the simulator's `trap()` method. We create 10 processes: p0, p1, ... p9. We stagger them to start from time 10 to 19.  Process p0 acts differently from the others: it sleeps for 5 and then triggers the trap. All the other processes, p1 to p9, simply wait on the trap. Inspect the results from running this example to see whether the processes behave as what you would expect.

#### 3.1.2  Barriers

It is rather straightforward to implement barriers using traps. A barrier is used to synchronize a group of processes. A barrier means that the process must stop at the barrier and cannot proceed until all other processes from the group reach this barrier. When the last process reaches the barrier, all processes can resume execution and continue from the barrier.

In the following example, we creates such a barrier.

In [10]:
# %load "../examples/basics/barrier.py"
import simulus

from random import seed, expovariate
seed(12345)

class Barrier(object):
    def __init__(self, sim, total_procs):
        self.sim = sim
        self.total_procs = total_procs
        # reset the barrier (by creating a new trap and 
        # resetting the count)
        self.trap = self.sim.trap()
        self.num = 0
        
    def barrier(self):
        self.num += 1
        if self.num < self.total_procs:
            # not yet all processes have reached the barrier
            self.trap.wait()
        else:
            # the last process has reached the barrier
            self.trap.trigger()
            # reset the barrier for next time use (by 
            # creating a new trap and resetting the count)
            self.trap = self.sim.trap()
            self.num = 0

def p(idx):
    while True:
        t = expovariate(1)
        print("p%d runs at %g and sleeps for %g" % (idx, sim.now, t))
        sim.sleep(t)
        print("p%d reaches barrier at %g" % (idx, sim.now))
        bar.barrier()
        
sim = simulus.simulator()
bar = Barrier(sim, 10)
for i in range(10):
    sim.process(p, i)
sim.run(10)


p0 runs at 0 and sleeps for 0.538916
p2 runs at 0 and sleeps for 0.0102212
p6 runs at 0 and sleeps for 1.74415
p9 runs at 0 and sleeps for 0.354734
p5 runs at 0 and sleeps for 0.459518
p1 runs at 0 and sleeps for 0.215251
p4 runs at 0 and sleeps for 0.83473
p3 runs at 0 and sleeps for 0.176365
p8 runs at 0 and sleeps for 0.132694
p7 runs at 0 and sleeps for 0.567284
p2 reaches barrier at 0.0102212
p8 reaches barrier at 0.132694
p3 reaches barrier at 0.176365
p1 reaches barrier at 0.215251
p9 reaches barrier at 0.354734
p5 reaches barrier at 0.459518
p0 reaches barrier at 0.538916
p7 reaches barrier at 0.567284
p4 reaches barrier at 0.83473
p6 reaches barrier at 1.74415
p6 runs at 1.74415 and sleeps for 0.825716
p2 runs at 1.74415 and sleeps for 0.191577
p8 runs at 1.74415 and sleeps for 0.805691
p3 runs at 1.74415 and sleeps for 0.438352
p1 runs at 1.74415 and sleeps for 3.17163
p9 runs at 1.74415 and sleeps for 0.0957338
p5 runs at 1.74415 and sleeps for 3.84624
p0 runs at 1.74415 and

The `Barrier` class implements the barrier. One creates a barrier with two arguments: the simulator on which the processes are run and the total number of processes for the barrier. We create a trap for the synchronization and a counter (call `num`) to keep track of the number of processes having reached the barrier. The counter is initialized to zero. Each time a process wants to use the barrier, it calls the `barrier()` method, which increments the counter. If the counter is smaller than the total number of processes, the process needs to wait (using the trap's `wait()` method) since the process is not the last one among the group to reach the barrier. Otherwise, if the counter gets to the total number of processes, the process is indeed the last one to reach the barrier. It calls the trap's `trigger()` method to release all the waiting processes that have earlier arrived at the barrier. Remember that traps are one-time use only. To make the barrier reusable, we create a new trap each time the barrier is reached by all processes. We also reset the counter.

In this example, we create 10 processes. Each process waits for some random time (exponentially distributed) and then calls for the barrier. The process then repeats forever.

### 3.2  Semaphores

A semaphore a multi-use signaling mechanism for inter-process communication. It is another primitive method beside trap designed for the simulation processes to synchronize and communicate with one another. A semaphore here implements what is commonly called a "counting semaphore." 

Initially, a semaphore can have a nonnegative integer count, which indicates the number of available resources. The processes atomically increment the semaphore count when resources are added or returned to the pool (using the `signal()` method) and atomically decrement the semaphore count when resources are removed (using the `wait()` method). When the semaphore count is zero, it means that there are no available resources. In that case, a process trying to decrement the semaphore (to remove a resource) will be blocked until more resources are added back to the pool. 

A semaphore is different from a trap. A trap is a one-time signaling mechanism. Multiple processes can wait on a trap. Once a process triggers the trap, *all* waiting processes will be unblocked. A trap cannot be reused: once a trap is sprung, subsequent waits will not block the processes and it cannot be triggered again. In comparison, a semaphore is a multi-use signaling mechanism. Each time a process waits on a semaphore, the semaphore value will be decremented. If the value becomes negative, the process will be blocked. Each time one signals a semaphore, the semaphore value will be incremented. If there are blocked processes, *one* of these processes will be unblocked. Processes can use the same semaphore repeatedly.

#### 3.2.1 Circular Wait

The following examples shows the use of semaphores to synchronize a group of processes.  

In [11]:
# %load "../examples/basics/circular.py"
import simulus

from random import seed, expovariate
seed(12345)

def p(idx):
    while True:
        sems[idx].wait()
        print("p%d wakes up at %g" % (idx, sim.now))
        sim.sleep(expovariate(1))
        sems[(idx+1)%total_procs].signal()
        
sim = simulus.simulator()

total_procs = 10
sems = [sim.semaphore() for _ in range(total_procs)]
for i in range(10):
    sim.process(p, i)
sems[0].signal()
sim.run(20)


p0 wakes up at 0
p1 wakes up at 0.538916
p2 wakes up at 0.549138
p3 wakes up at 2.29329
p4 wakes up at 2.64802
p5 wakes up at 3.10754
p6 wakes up at 3.32279
p7 wakes up at 4.15752
p8 wakes up at 4.33388
p9 wakes up at 4.46658
p0 wakes up at 5.03386
p1 wakes up at 5.85958
p2 wakes up at 6.05115
p3 wakes up at 6.85685
p4 wakes up at 7.2952
p5 wakes up at 10.4668
p6 wakes up at 10.5626
p7 wakes up at 14.4088
p8 wakes up at 14.94
p9 wakes up at 15.6411
p0 wakes up at 15.8014
p1 wakes up at 17.0707
p2 wakes up at 17.2814
p3 wakes up at 17.6993
p4 wakes up at 17.7231
p5 wakes up at 18.1379


In this example, the processes are organized into a circle. Each process is created with a semaphore; it waits for the semaphore and then signal the semaphore of the subsequent process, and then repeats. In this case, all the processes are executed one at a time in a round robin fashion.

The use of the semaphore in this case is very much like a trap, since there is at most one process waiting at a time. Whether the semaphore unblocks one waiting process at a time, or the trap unblocks all waiting processes at a time does not really matter in this case. However, a semaphore can be reused, while a trap can only be used once. If we use traps for this example, we would have to create a new trap each time a process wakes up.

#### 3.2.2  Producer-Consumer Problem

If we learn from our Operation Systems class, we should have known about the producer–consumer problem (also known as the bounded-buffer problem). It is a classic problem for multi-process synchronization. In its simpliest form, the problem consists of two processes. A producer process repeatedly generates data and put them into a common, fixed sized buffer. A consumer consumes the data by removing them from the buffer one at a time. The problem is that the producer cannot put data into the buffer if the buffer is already full. Similarly, the consumer cannot remove data from the buffer if the buffer is empty.

One solution to the producer-consumer problem is to use semaphores, as shown in the following example. Later we will show an even simpler way to solve this problem (using simulus provided `resource` module).

In [12]:
# %load "../examples/basics/boundbuf.py"
import simulus

from random import seed, expovariate, gauss
seed(12345)

bufsiz = 5 # buffer capacity
items_produced = 0 # keep track the number of items produced
items_consumed = 0 # ... and consumed
num_producers = 2 # number of producers
num_consumers = 3 # number of consumers

def producer(idx):
    global items_produced
    while True:
        sim.sleep(expovariate(1)) # take time to produce an item
        num = items_produced
        items_produced += 1
        print("%f: p[%d] produces item [%d]" % (sim.now, idx, num))
        sem_avail.wait() # require a free slot in buffer
        sem_occupy.signal() # store the item and increase occupancy
        print("%f: p[%d] stores item [%d] in buffer" % 
              (sim.now, idx, num))
        
def consumer(idx):
    global items_consumed
    while True:
        sem_occupy.wait() # require an item from buffer
        sem_avail.signal() # retrieve the item and bump the free slots
        num = items_consumed
        items_consumed += 1
        print("%f: c[%d] retrieves item [%d] from buffer" %
              (sim.now, idx, num))
        sim.sleep(gauss(0.8, 0.2)) # take time to consume the item
        print("%f: c[%d] consumes item[%d]" % (sim.now, idx, num))        

sim = simulus.simulator()
sem_avail = sim.semaphore(bufsiz) # available slots
sem_occupy = sim.semaphore(0) # no items yet
for i in range(num_producers): 
    sim.process(producer, i)
for i in range(num_consumers):
    sim.process(consumer, i)
sim.run(10)


0.010221: p[1] produces item [0]
0.010221: p[1] stores item [0] in buffer
0.010221: c[0] retrieves item [0] from buffer
0.538916: p[0] produces item [1]
0.538916: p[0] stores item [1] in buffer
0.538916: c[2] retrieves item [1] from buffer
0.752533: c[0] consumes item[0]
0.754168: p[0] produces item [2]
0.754168: p[0] stores item [2] in buffer
0.754168: c[1] retrieves item [2] from buffer
1.521765: c[2] consumes item[1]
1.588897: p[0] produces item [3]
1.588897: p[0] stores item [3] in buffer
1.588897: c[0] retrieves item [3] from buffer
1.608449: c[1] consumes item[2]
1.754371: p[1] produces item [4]
1.754371: p[1] stores item [4] in buffer
1.754371: c[2] retrieves item [4] from buffer
2.156181: p[0] produces item [5]
2.156181: p[0] stores item [5] in buffer
2.156181: c[1] retrieves item [5] from buffer
2.476470: c[0] consumes item[3]
2.580087: p[1] produces item [6]
2.580087: p[1] stores item [6] in buffer
2.580087: c[0] retrieves item [6] from buffer
2.594533: p[0] produces item [7]

We use two semaphores: one semaphore `sem_avail` is used to count the number of free slots in the buffer, and the other semaphore `sem_occupy` is used to count the number of produced items stored in the buffer. 

A producer sleeps for some random time which is exponentially distributed to represent the production of an item, then calls `wait()` on the semaphore `sem_avail` to decrement the available slots in the buffer. The process may be blocked if there is no more free slots available, in which case we wait for a consumer to retrieve an item and thereby creates a free slot. Otherwise, the producer adds the item into the buffer, which is represented by calling `signal()` on the semaphore `sem_occupy`, which increments the number of occupied items in the buffer. The producer then repeats.

A consumer calls `wait()` on the semaphore `sem_occupy` to decrement the number of items stored in the buffer. The process may be blocked if there are currently no items in the buffer. In this case, the process will wait until a producer adds an item. Otherwise, the consumer retrieves the item, which is represented by calling `signal()` on teh semaphore `sem_avail`, which increments the number of available slots in the buffer. The consumer then consumes the item, by randomly waiting for some random time, which is normally distributed. The consumer then repeats.

In this example, we create two producer and three consumer processes, and use the two semaphores to synchronize the processes to access the bounded buffer. If you're familiar with the performance modeling and simulation, you may recognize that we are actually simulating a multi-server queue in this case with limited capacity.

#### 3.2.3  Queuing Disciplines

If multiple processes are waiting on a semaphore, it uses a FIFO order (first in first out) by default. That is, the first process gets blocked on the semaphore will be unblocked first. 

Other queuing disciplines are also possible, including LIFO (last in first out), RANDOM, and PRIORITY (depending on the 'priority' of the processes, where a lower value means higher priority). One can choose a queuing discipline when the semaphore is created. The queuing disciplines are constants defined in the `QDIS` class.

In the following example, we show the use of different queuing disciplines.

In [13]:
# %load "../examples/basics/qdis.py"
import simulus

# so that we get same result from random priority
from random import seed
seed(12345)

def p(idx, sem):
    # set the priority of the current process (this is only useful 
    # if we use PRIORITY qdis)
    sim.set_priority(abs(idx-3.2))

    # make sure the process wait on the semaphore in order
    sim.sleep(idx)

    # the process will block on the semaphore and then print out 
    # a message when it is unblocked
    sem.wait()
    print("p[id=%d,prio=%.1f] resumes at %f" % 
          (idx, sim.get_priority(), sim.now))

def trywaits(sem):
    # create ten processes which will all block on the semaphore
    for i in range(10):
        sim.process(p, idx=i, sem=sem)
    sim.sleep(100)
    
    # release them all and check the order they are unblocked
    print('-'*40)
    for i in range(10):
        sem.signal()

sim = simulus.simulator()
s1 = sim.semaphore()
s2 = sim.semaphore(qdis=simulus.QDIS.LIFO)
s3 = sim.semaphore(qdis=simulus.QDIS.RANDOM)
s4 = sim.semaphore(qdis=simulus.QDIS.PRIORITY)
sim.process(trywaits, s1, offset=0)
sim.process(trywaits, s2, offset=100)
sim.process(trywaits, s3, offset=200)
sim.process(trywaits, s4, offset=300)
sim.run()


----------------------------------------
p[id=0,prio=3.2] resumes at 100.000000
p[id=1,prio=2.2] resumes at 100.000000
p[id=2,prio=1.2] resumes at 100.000000
p[id=3,prio=0.2] resumes at 100.000000
p[id=4,prio=0.8] resumes at 100.000000
p[id=5,prio=1.8] resumes at 100.000000
p[id=6,prio=2.8] resumes at 100.000000
p[id=7,prio=3.8] resumes at 100.000000
p[id=8,prio=4.8] resumes at 100.000000
p[id=9,prio=5.8] resumes at 100.000000
----------------------------------------
p[id=9,prio=5.8] resumes at 200.000000
p[id=8,prio=4.8] resumes at 200.000000
p[id=7,prio=3.8] resumes at 200.000000
p[id=6,prio=2.8] resumes at 200.000000
p[id=5,prio=1.8] resumes at 200.000000
p[id=4,prio=0.8] resumes at 200.000000
p[id=3,prio=0.2] resumes at 200.000000
p[id=2,prio=1.2] resumes at 200.000000
p[id=1,prio=2.2] resumes at 200.000000
p[id=0,prio=3.2] resumes at 200.000000
----------------------------------------
p[id=5,prio=1.8] resumes at 300.000000
p[id=1,prio=2.2] resumes at 300.000000
p[id=2,prio=1.2] re

We create four semaphores, one for each queuing discipline. This is achieved by passing the `qdis` argument to the simulator's `semaphore()` method for creating a semaphore. To use the priority based queuing discipline, we set the current process's priority using the simulator's `set_priority()` method. 

For each of the four semaphores, we create a `trywaits` process, which then creates another 10 processes who will wait on the semaphore. Then the `trywaits` process will release all the waiting processes. From the print-out, we should be able to determine the order in which the waiting processes are unblocked. 

### 3.3  Trappables and Timed Wait

Both traps and semaphores are called trappables. Simulus provides a very powerful function in simulator called `wait()` to block the calling process until any or all of the trappables are triggered, or until a pre-specified amount of time, if provided, has elapsed. 

If we have only one trap, say `t`, calling the wait method of the trap, `t.wait()`, is equivalent to calling the simulator's wait function with the trap as the argument: `sim.wait(t)`. Similarly, if we have only one semaphore, say `s`, `s.wait()` would be equivalent to `sim.wait(s)`. In this case, we pass the trap `t` or semaphore `s` to the simulator's `wait()` function, since they are trappables.

The simulator's `wait()` function actually can take more than one trappables in a list or tuple. In this case, the calling process can wait until either one of the trappables, or all of the trappables are triggered, which depends on the `method` argument: if it's `any`, the wait condition is satisfied as soon as one of the trappables is triggered; if it's 'all', the process needs to wait until all trappables are triggered. If the method argument is not provided, simulus assumes it's all.

#### 3.3.1  Waiting on Multiple Trappables

The following shows an example of a process waiting for multiple trappables.

In [14]:
# %load "../examples/basics/multiwait.py"
import simulus

def p1():
    sim.sleep(10)
    print("p1 triggers t1 at %g" % sim.now)
    t1.trigger()

    sim.sleep(10)
    print("p1 triggers s1 at %g" % sim.now)
    s1.trigger() # signal and trigger are aliases for semaphore

    sim.sleep(10)
    print("p1 triggers t2 at %g" % sim.now)
    t2.trigger()

    sim.sleep(10)
    print("p1 triggers s2 at %g" % sim.now)
    s2.signal()

def p2():
    tp = (t1, s1)
    r = sim.wait(tp)
    print("p2 resumes at %g (ret=%r)" % (sim.now, r))

    tp = [t2, s2]
    r = sim.wait(tp, method=any)
    print("p2 resumes at %g (ret=%r)" % (sim.now, r))

    # find the remaining untriggered trappables (using the 
    # returned mask) and wait for them all
    tp = [t for i, t in enumerate(tp) if not r[i]]
    r = sim.wait(tp)
    print("p2 resumes at %g (ret=%r)" % (sim.now, r))

sim = simulus.simulator()
t1 = sim.trap()
t2 = sim.trap()
s1 = sim.semaphore()
s2 = sim.semaphore()
sim.process(p1)
sim.process(p2)
sim.run()


p1 triggers t1 at 10
p1 triggers s1 at 20
p2 resumes at 20 (ret=([True, True], False))
p1 triggers t2 at 30
p2 resumes at 30 (ret=([True, False], False))
p1 triggers s2 at 40
p2 resumes at 40 (ret=([True], False))


In this example, we create two traps, t1 and t2, and two semaphores, s1 and s2. We also create two processes. The process p1 triggers a trap or a semaphore at 10 seconds interval. The other process p2 first waits on a trap and a semaphore using the default method (`all`). The process will only resume execution once both trappables are triggered. The process then waits on the other trap and semaphore using the `any` method. In this case, when one trappable is triggered, the process will resume execution. It then filters out the triggered trappables from the list and creates another list with the remaining untriggered trappables (actually only one left), and then wait until all of the remaining are triggered.

The return from the `wait()` function needs further explanation. The function actually returns a tuple with two elements: the first element of the tuple is to indicate whether the trappables have been triggered or not; and the second element of tuple is to indicate whether timeout happens. Since in this example, we don't use timed wait, the `wait()` function always returns False in the second element.

If the first argument when calling the `wait()` function is only one trappable (not in a list or tuple), the first element of the returned tuple will simply be True or False, depending on whether the trappable has been triggered or not upon the return of the function. If, on the other hand, the first argument when calling the `wait()` functionis a list or a tuple of trappables (even if with only one trappable), the first element of the returned tuple will be a list of booleans, where each element of the list indicates whether the corresponding trappable has been triggered or not.

In the example, when the `wait()` function is called the first time, it returns a list with True and True for the first element of the returned tuple, since both trappables need to be triggered before the process can resume execution (because of the method `all`). In the second time, the function returns True and False. Only the first trappable (the trap t2) is triggered at the time (because of the method `any`). In the third time, the remaining trappable (the semaphore s2) is triggered and the function returns a list consisted of only one element being True.

We can optionally provide either an 'offset' or 'until' argument to the `wait()` function. The 'offset' is the relative time from now until which the process will be put on hold at the latest; if provided, it must be a non-negative value. The 'until' is the absolute time at which the process is expected to resume execution at the latest; if provided, it must not be earlier than the current time; either 'offset' or 'until' can be provided, but not both. If both 'offset' and 'until' are ignored (like what we have previously), there will be no time limit on the wait.

#### 3.3.2  Race between Tom and Jerry

To be able to perform timed wait on multiple trappables allows a process-oriented model to be quite expressive. The following shows a good example. Tom and Jerry decides to enter a race. We model Tom as a process. Each time, Tom calls `sleep()` to represent running for some time, which is a random variable from a normal distribution with a mean 100 and a standard deviation of 50. We model Jerry as an event. Jerry calls `sched()` to schedule an event to represent running for some time, which is a random variable from a uniform distribution between 50 and 100. Tom and Jerry compete for ten times; the next race starts as soon as the previous one finishes. Whoever runs fastest wins. But if they run for more than 100 seconds, both are disqualified.

In [15]:
# %load "../examples/basics/tomjerry.py"
import simulus

from random import seed, gauss, uniform
seed(321)

def tom():
    sim.sleep(gauss(100, 50))
    print("%g: tom finished" % sim.now)

def jerry():
    print("%g: jerry finished" % sim.now)

def compete():
    for _ in range(10):
        print("competition starts at %g -->" % sim.now)

        p = sim.process(tom) # run, tom, run!
        e = sim.sched(jerry, offset=uniform(50, 150)) # run, jerry, run!
    
        # let the race begin...
        (r1, r2), timedout = sim.wait((p, e), 100, method=any)
        if timedout:
            print("%g: both disqualified" % sim.now)
            sim.kill(p)
            sim.cancel(e)
        elif r1: 
            print("%g: tom wins" % sim.now)
            sim.cancel(e)
        else:
            print("%g: jerry wins" % sim.now)
            sim.kill(p)

sim = simulus.simulator()
sim.process(compete)
sim.run()


competition starts at 0 -->
77.5459: jerry finished
77.5459: jerry wins
competition starts at 77.5459 -->
171.749: jerry finished
171.749: jerry wins
competition starts at 171.749 -->
271.749: both disqualified
competition starts at 271.749 -->
357.072: tom finished
357.072: tom wins
competition starts at 357.072 -->
430.387: tom finished
430.387: tom wins
competition starts at 430.387 -->
485.297: tom finished
485.297: tom wins
competition starts at 485.297 -->
585.297: both disqualified
competition starts at 585.297 -->
611.838: tom finished
611.838: tom wins
competition starts at 611.838 -->
711.838: both disqualified
competition starts at 711.838 -->
811.838: both disqualified


Earlier, we showed traps and semaphores as trappables that we can perform conditional and timed wait. In this example, we show that both events and processes are also trappables. An instance of the process is returned from the `process()` function, and an instance of the event is returned from the `sched()` function. Both are both considered as opaque objects. That is, the users are not expected to inspect the content of a process or event. Rather, the users should simple use the references. A process is a trappable, which is triggered when the process terminates. An event is also a trappable, which is triggered upon the activation of the event (i.e., when the event is processed).

In this example, we call the simulator's `wait()` function to wait for `any` of the trappables to be triggered. We also provide a time limit of 100 (we use 'offset' as a positional argument). We look at the return value from the `wait()` function to determine the outcome of the race. If the wait is timed out, we both `kill()` the process and `cancel()` the event. Otherwise, if Tom wins (when p is triggered), we just cancel the event; and if Jerry wins (when e is triggered), we instead kill the process.

## 4. Resources and Facilities

In the previous section, we discussed traps and semaphores, which are the two primitive methods in simulus for process synchronization. Simulus provides several additional advanced mechanisms to coordinate processes and facilitate inter-process communication to ease model development. 

In [16]:
# %load "../examples/basics/mm1.py"
import simulus

from random import seed, expovariate
seed(123)

def job(idx):
    r.acquire()
    print("%g: job(%d) gains access " % (sim.now,idx))
    sim.sleep(expovariate(1))
    print("%g: job(%d) releases" % (sim.now,idx))
    r.release()
    
def arrival():
    i = 0
    while True:
        i += 1
        sim.sleep(expovariate(0.95))
        print("%g: job(%d) arrives" % (sim.now,i))
        sim.process(job, i)

sim = simulus.simulator()
r = sim.resource()
sim.process(arrival)
sim.run(10)


0.0566152: job(1) arrives
0.0566152: job(1) gains access 
0.15264: job(2) arrives
0.272591: job(3) arrives
0.579584: job(1) releases
0.579584: job(2) gains access 
0.618484: job(2) releases
0.618484: job(3) gains access 
1.38679: job(3) releases
2.70906: job(4) arrives
2.70906: job(4) gains access 
3.13407: job(5) arrives
3.31718: job(6) arrives
3.75014: job(7) arrives
4.17767: job(8) arrives
4.47373: job(9) arrives
4.47549: job(10) arrives
4.62019: job(4) releases
4.62019: job(5) gains access 
4.71188: job(5) releases
4.71188: job(6) gains access 
5.07885: job(11) arrives
5.1551: job(12) arrives
5.55405: job(13) arrives
5.62219: job(6) releases
5.62219: job(7) gains access 
6.18015: job(14) arrives
6.28263: job(15) arrives
6.44405: job(16) arrives
7.98027: job(7) releases
7.98027: job(8) gains access 
8.00174: job(8) releases
8.00174: job(9) gains access 
8.0872: job(17) arrives
8.98396: job(18) arrives
9.30851: job(19) arrives


In [17]:
# %load "../examples/basics/bank.py"
import simulus

# this example is modified from the simpy's bank renege example; we
# use the same settings so that we can get the same results
RANDOM_SEED = 42        # random seed for repeatability
NUM_CUSTOMERS = 5       # total number of customers
INTV_CUSTOMERS = 10.0   # mean time between new customers
MEAN_BANK_TIME = 12.0   # mean time in bank for each customer
MIN_PATIENCE = 1        # min customer patience
MAX_PATIENCE = 3        # max customer patience

from random import seed, expovariate, uniform

def source():
    for i in range(NUM_CUSTOMERS):
        sim.process(customer, i)
        sim.sleep(expovariate(1.0/INTV_CUSTOMERS))

def customer(idx):
    arrive = sim.now
    print('%7.4f Customer%02d: Here I am' % (arrive, idx))

    patience = uniform(MIN_PATIENCE, MAX_PATIENCE)
    _, timedout = sim.wait(counter, patience)
    if timedout:
        print('%7.4f Customer%02d: RENEGED after %6.3f' %
              (sim.now, idx, sim.now-arrive))
    else:
        # We got to the counter
        print('%7.4f Customer%02d: Waited %6.3f' %
              (sim.now, idx, sim.now-arrive))
        sim.sleep(expovariate(1.0/MEAN_BANK_TIME))
        print('%7.4f Customer%02d: Finished' % (sim.now, idx))
        counter.release()

print('Bank renege')
seed(RANDOM_SEED)
sim = simulus.simulator()
counter = sim.resource()
sim.process(source)
sim.run()


Bank renege
 0.0000 Customer00: Here I am
 0.0000 Customer00: Waited  0.000
 3.8595 Customer00: Finished
10.2006 Customer01: Here I am
10.2006 Customer01: Waited  0.000
12.7265 Customer02: Here I am
13.9003 Customer02: RENEGED after  1.174
23.7507 Customer01: Finished
34.9993 Customer03: Here I am
34.9993 Customer03: Waited  0.000
37.9599 Customer03: Finished
40.4798 Customer04: Here I am
40.4798 Customer04: Waited  0.000
43.1401 Customer04: Finished
